In [1]:
import pandas as pd
import random 
import itertools
import json
import matplotlib.pyplot as plt
import os

pd.set_option('display.max_rows', 50)


from functions import sharpe_ratio_calculation, generate_rand_portfolios, select_top_five, join_stocks_crypto, test_for_silhouette_score, run_clustering_model, label_balance

c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
df_all_stocks = pd.read_csv('stocks_data_filled.csv',index_col='Date')
cryptos_df = pd.read_csv('cryptos_data.csv', index_col='Date')

joined_df = join_stocks_crypto(cryptos_df, df_all_stocks, mode = 'stocks_left') #mode - either do left with crypto and fill NA for stocks or do left on stocks and leave out some dates for cryptos
joined_df.index = pd.to_datetime(joined_df.index)
joined_df = joined_df.bfill()

joined_df_weekly = joined_df.resample('W').last() #try aggregating on a weekly level
joined_df_3days = joined_df.resample('3D').last()# aggregating on a twice per week basis to arrive at the sweet spot of that 250 (1 year) timeseries length

In [3]:
#Random Portfolios generation
tickers = list(df_all_stocks.columns)

random.seed(42)
random_portfolios = generate_rand_portfolios(n_reps=1000, n_stocks=15, tickers=tickers)

In [ ]:
random_portfolios

In [4]:
#Select top five sharpe ratio portfolios from a portfolio
sharpe_ratio = sharpe_ratio_calculation(df_all_stocks, rf_rate_annual = 0.02)
top_five_dict = select_top_five(random_portfolios, metric=sharpe_ratio)

In [5]:
#Optimize Traditional Portfolios
# for i in range(999, 1001):

#     print('Doing', i)

#     top_five_sets = dict(itertools.islice(top_five_dict.items(), i, i+1))
#     results = run_min_variance(df_all_stocks, top_five_sets, min_weight_for_top_five=0.05)  #TRY DIFFERENT WEIGHTS FOR top_five
#     with open(f"min_variance_portfolio_jsons/my_dict{i}.json", "w") as f:
#         json.dump(results, f, indent=4)




#Reassemble the results of the optimization - jsons
# min_var_portfolios = dict()
# for i in range(1,1000):
#     with open(f'min_variance_portfolio_jsons/my_dict{i}.json') as f:
#         port = json.load(f)
#         min_var_portfolios.update(port)

# with open(f"full_optimized_min_variance.json", "w") as f:
#     json.dump(min_var_portfolios, f, indent=4)

In [6]:
#RUN THE CLUSTERING WITH DIFFERENT SET UPS TO GET THE SILHOUETTE SCORES FOR COMPARISON

# n_clusters_list = [4,5,6,7]
# linkage_list=['single', 'average', 'complete']
# #window_sizes = [3,7,10,14,21,30,60]
# window_sizes = [21,30,60]


# def run_clustering_evaluation(df, window_sizes, method, moving_average=True, return_mode='arithmetic', df_input_name='DFWASNOTSPECIFIED'):

#     for w_size in window_sizes:

#         #return_mode = 'arithmetic'
#         #n_init = 3
#         #center = True
#         if moving_average:
#             df = df.rolling(window=w_size, center=True).mean()

#             smoothing = 'moving_average'
#         else:
#             smoothing = 'no_smoothing'

#         if len(df) < 150 and w_size > 30:
#             continue
#         silhouette_df = test_for_silhouette_score(df, n_clusters_list, method=method, return_mode=return_mode, n_init=3, linkage_list=linkage_list)

#         silhouette_df['return_mode'] = return_mode
#         silhouette_df['n_init'] = 3
#         silhouette_df['smoothing'] = smoothing
#         silhouette_df['window_size/span'] = w_size

#         silhouette_df.to_csv(f'silhouette_dfs/{method}_{smoothing}_{return_mode}_windowsize-{w_size}_{df_input_name}.csv')




# df_input_name = input('Put in the name of the df mode you are running for: ')
# for return_mode in ['arithmetic', 'geometric']:
#     run_clustering_evaluation(joined_df_3days, window_sizes, method='kmeans', moving_average=True, return_mode=return_mode, df_input_name=df_input_name)

In [ ]:
#RUN FOR LABEL BALANCE

method_loop = 'kmeans'
return_mode_loop = 'geometric'
n_clus_loop = 7

df_with_label_balance = pd.DataFrame()
for df_dict in [{'weekly': joined_df_weekly}, {'3day': joined_df_3days}, {'full': joined_df}]:
    for linkage in ['single', 'complete', 'average']:
        for w in [3,7,10,14,30]:
            output = label_balance(df_dict, w, method_loop, return_mode_loop, n_clus=n_clus_loop, linkage=linkage)
            df_with_label_balance = pd.concat([df_with_label_balance, output])

df_with_label_balance.reset_index(inplace=True)

df_with_label_balance.to_csv(f'balance_datasets/balance_test_results_{method_loop}_{return_mode_loop}_{n_clus_loop}clusters.csv')

In [41]:
def rejoin_dfs(folder_path, extract_cols_from_filename=False):
    '''
    extract_cols_from_filename is the option to handle legacy thing with the silhouette score tables when part of the information was stored in the filename
    '''
    dfs = pd.DataFrame()
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)  
        df = pd.read_csv(file_path, index_col=False)
        if extract_cols_from_filename:
            df['filename'] = filename
        dfs = pd.concat([dfs, df])
    for col in ['index', 'Unnamed: 0']:
        try:
            dfs = dfs.drop(columns=[col])
        except:
            pass
    if extract_cols_from_filename:
        dfs['df_mode'] = dfs['filename'].apply(lambda x: x.split('-')[1].split('_')[1].replace('.csv', ''))
        dfs['return_mode'] = dfs['filename'].apply(lambda x: x.split('_')[3])
        dfs['linkage'] = dfs['linkage'].fillna('not_applicable')

    return dfs

In [42]:
folder_path_sil = 'silhouette_dfs/'
folder_path_balance = 'balance_datasets/'

silhouette_results = rejoin_dfs(folder_path_sil, extract_cols_from_filename=True)
balance_results = rejoin_dfs(folder_path_balance)


In [43]:
balance_results

,return_mode,window_size,method,linkage,df_mode,min_per_cluster,max_per_cluster,min_max_delta,filename,clusters
0,arithmetic,3,ahc,single,weekly,0.0045,0.9685,0.9640,balance_test_results_ahc_arithmetic_5clusters.csv,5
1,arithmetic,7,ahc,single,weekly,0.0045,0.9640,0.9595,balance_test_results_ahc_arithmetic_5clusters.csv,5
2,arithmetic,10,ahc,single,weekly,0.0045,0.9550,0.9505,balance_test_results_ahc_arithmetic_5clusters.csv,5
3,arithmetic,14,ahc,single,weekly,0.0045,0.9550,0.9505,balance_test_results_ahc_arithmetic_5clusters.csv,5
4,arithmetic,30,ahc,single,weekly,0.0045,0.9550,0.9505,balance_test_results_ahc_arithmetic_5clusters.csv,5
...,...,...,...,...,...,...,...,...,...,...
40,geometric,3,kshape,not_applicable,full,0.0541,0.3874,0.3333,balance_test_results_kshape_geometric_6cluster...,6
41,geometric,7,kshape,not_applicable,full,0.0811,0.2432,0.1622,balance_test_results_kshape_geometric_6cluster...,6
42,geometric,10,kshape,not_applicable,full,0.0811,0.2973,0.2162,balance_test_results_kshape_geometric_6cluster...,6
43,geometric,14,kshape,not_applicable,full,0.0811,0.3243,0.2432,balance_test_results_kshape_geometric_6cluster...,6


In [46]:
silhouette_results

,clusters,silhouette_score,method,linkage,return_mode,n_init,smoothing,window_size/span,filename,inertia,df_mode
0,4,0.450849,ahc,single,arithmetic,3,moving_average,10,ahc_moving_average_arithmetic_windowsize-10_3d...,NaN,3day
1,5,0.366116,ahc,single,arithmetic,3,moving_average,10,ahc_moving_average_arithmetic_windowsize-10_3d...,NaN,3day
2,6,0.217847,ahc,single,arithmetic,3,moving_average,10,ahc_moving_average_arithmetic_windowsize-10_3d...,NaN,3day
3,7,0.186290,ahc,single,arithmetic,3,moving_average,10,ahc_moving_average_arithmetic_windowsize-10_3d...,NaN,3day
4,4,0.436554,ahc,average,arithmetic,3,moving_average,10,ahc_moving_average_arithmetic_windowsize-10_3d...,NaN,3day
...,...,...,...,...,...,...,...,...,...,...,...
3,7,0.175246,kshape,not_applicable,geometric,3,moving_average,7,kshape_moving_average_geometric_windowsize-7_f...,0.150242,full
0,4,0.122094,kshape,not_applicable,geometric,3,moving_average,7,kshape_moving_average_geometric_windowsize-7_w...,0.114208,weekly
1,5,0.109347,kshape,not_applicable,geometric,3,moving_average,7,kshape_moving_average_geometric_windowsize-7_w...,0.103348,weekly
2,6,0.106075,kshape,not_applicable,geometric,3,moving_average,7,kshape_moving_average_geometric_windowsize-7_w...,0.101805,weekly


In [47]:
df_all_results = pd.merge(silhouette_results, balance_results, how='left', 
                          left_on=['clusters', 'linkage', 'window_size/span', 'df_mode', 'method', 'return_mode'],
                          right_on=['clusters', 'linkage', 'window_size', 'df_mode', 'method', 'return_mode'])

In [51]:
df_all_results.drop(columns=['filename_x', 'filename_y'])

,clusters,silhouette_score,method,linkage,return_mode,n_init,smoothing,window_size/span,inertia,df_mode,window_size,min_per_cluster,max_per_cluster,min_max_delta
0,4,0.450849,ahc,single,arithmetic,3,moving_average,10,NaN,3day,NaN,NaN,NaN,NaN
1,5,0.366116,ahc,single,arithmetic,3,moving_average,10,NaN,3day,10.0,0.0045,0.9550,0.9505
2,6,0.217847,ahc,single,arithmetic,3,moving_average,10,NaN,3day,10.0,0.0045,0.9505,0.9459
3,7,0.186290,ahc,single,arithmetic,3,moving_average,10,NaN,3day,NaN,NaN,NaN,NaN
4,4,0.436554,ahc,average,arithmetic,3,moving_average,10,NaN,3day,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5,0.109347,kshape,not_applicable,geometric,3,moving_average,7,0.103348,weekly,7.0,0.0586,0.2793,0.2207
1036,6,0.106075,kshape,not_applicable,geometric,3,moving_average,7,0.101805,weekly,7.0,0.1036,0.2613,0.1577
1037,6,0.106075,kshape,not_applicable,geometric,3,moving_average,7,0.101805,weekly,7.0,0.0676,0.2342,0.1667
1038,6,0.106075,kshape,not_applicable,geometric,3,moving_average,7,0.101805,weekly,7.0,0.0856,0.2387,0.1532
